## **0. Подготовка окружения**




In [21]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
# Путь к папке с данными
DATASET_PATH = "./data"  # внутри: папки 1,2,...,8

# !pip install mediapipe opencv-python scikit-learn

In [23]:
import os
import cv2
import numpy as np
import mediapipe as mp
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# from google.colab.patches import cv2_imshow
import random

mp_pose = mp.solutions.pose
pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

# Добавляем FaceMesh для эмоций
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,  # включает landmarks для глаз и губ
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)


I0000 00:00:1766780420.681324 12037841 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Pro
I0000 00:00:1766780420.697587 12037841 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 90.5), renderer: Apple M1 Pro


In [24]:
def extract_face_features(rgb_frame):
    """
    Извлекает признаки эмоций из лица используя FaceMesh.
    """
    result = face_mesh.process(rgb_frame)
    N_FACE_FEATURES = 20
    
    if not result.multi_face_landmarks:
        return np.zeros(N_FACE_FEATURES)
    
    face_landmarks = result.multi_face_landmarks[0].landmark
    nose = face_landmarks[1]
    left_eye = face_landmarks[33]
    right_eye = face_landmarks[263]
    eye_dist = max(np.sqrt((left_eye.x - right_eye.x)**2 + (left_eye.y - right_eye.y)**2), 0.01)
    
    features = []
    # Открытость глаз
    features.extend([(face_landmarks[159].y - face_landmarks[145].y) / eye_dist,
                     (face_landmarks[386].y - face_landmarks[374].y) / eye_dist])
    # Положение бровей
    features.extend([(face_landmarks[105].y - face_landmarks[159].y) / eye_dist,
                     (face_landmarks[334].y - face_landmarks[386].y) / eye_dist])
    # Рот
    mouth_open = (face_landmarks[14].y - face_landmarks[13].y) / eye_dist
    mouth_width = (face_landmarks[308].x - face_landmarks[78].x) / eye_dist
    features.extend([mouth_open, mouth_width])
    mouth_center_y = (face_landmarks[13].y + face_landmarks[14].y) / 2
    features.extend([(mouth_center_y - face_landmarks[78].y) / eye_dist,
                     (mouth_center_y - face_landmarks[308].y) / eye_dist])
    # Голова
    features.extend([(left_eye.y - right_eye.y) / eye_dist,
                     (nose.x - (left_eye.x + right_eye.x) / 2) / eye_dist])
    features.append((face_landmarks[13].y - nose.y) / eye_dist)
    for idx in [33, 263, 61, 291, 199, 175, 152, 10, 234]:
        features.append((face_landmarks[idx].x - nose.x) / eye_dist)
    
    return np.array(features[:N_FACE_FEATURES])


def extract_pose_vector(frame):
    """
    Принимает BGR-кадр, возвращает вектор позы + лица + признаки рук.
    """
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(rgb)

    if not result.pose_landmarks:
        return None

    landmarks = result.pose_landmarks.landmark
    
    LEFT_SHOULDER, RIGHT_SHOULDER = 11, 12
    LEFT_HIP, RIGHT_HIP = 23, 24
    LEFT_WRIST, RIGHT_WRIST = 15, 16
    
    center_x = (landmarks[LEFT_SHOULDER].x + landmarks[RIGHT_SHOULDER].x + 
                landmarks[LEFT_HIP].x + landmarks[RIGHT_HIP].x) / 4
    center_y = (landmarks[LEFT_SHOULDER].y + landmarks[RIGHT_SHOULDER].y + 
                landmarks[LEFT_HIP].y + landmarks[RIGHT_HIP].y) / 4
    
    shoulder_dist = max(np.sqrt(
        (landmarks[LEFT_SHOULDER].x - landmarks[RIGHT_SHOULDER].x) ** 2 +
        (landmarks[LEFT_SHOULDER].y - landmarks[RIGHT_SHOULDER].y) ** 2
    ), 0.01)

    vec = []
    for lm in landmarks:
        norm_x = (lm.x - center_x) / shoulder_dist
        norm_y = (lm.y - center_y) / shoulder_dist
        vec.extend([norm_x, norm_y, lm.visibility])

    # Явные признаки видимости и положения рук
    left_wrist_vis = landmarks[LEFT_WRIST].visibility
    right_wrist_vis = landmarks[RIGHT_WRIST].visibility
    left_hand_visible = 1.0 if left_wrist_vis > 0.5 else 0.0
    right_hand_visible = 1.0 if right_wrist_vis > 0.5 else 0.0
    any_hand_visible = 1.0 if (left_wrist_vis > 0.5 or right_wrist_vis > 0.5) else 0.0
    left_wrist_above_shoulder = 1.0 if landmarks[LEFT_WRIST].y < landmarks[LEFT_SHOULDER].y else 0.0
    right_wrist_above_shoulder = 1.0 if landmarks[RIGHT_WRIST].y < landmarks[RIGHT_SHOULDER].y else 0.0
    left_hand_dist = np.sqrt((landmarks[LEFT_WRIST].x - center_x)**2 + 
                             (landmarks[LEFT_WRIST].y - center_y)**2) / shoulder_dist
    right_hand_dist = np.sqrt((landmarks[RIGHT_WRIST].x - center_x)**2 + 
                              (landmarks[RIGHT_WRIST].y - center_y)**2) / shoulder_dist
    
    vec.extend([left_hand_visible, right_hand_visible, any_hand_visible,
                left_wrist_above_shoulder, right_wrist_above_shoulder,
                left_hand_dist, right_hand_dist])

    # Признаки лица
    face_features = extract_face_features(rgb)
    vec.extend(face_features)

    return np.array(vec)


W0000 00:00:1766780420.706244 12098312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766780420.720864 12098312 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [25]:
print("Содержимое DATASET_PATH:")
!ls "$DATASET_PATH"


W0000 00:00:1766780420.857357 12098301 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1766780420.892164 12098305 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Содержимое DATASET_PATH:
1         3         5         7         model.pkl
2         4         6         8


In [26]:
from torchvision import transforms
from PIL import Image

# Создаем трансформер
augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),      # Отражаем слева направо
    transforms.RandomRotation(degrees=15),       # Крутим на +- 15 градусов
    transforms.ColorJitter(brightness=0.2),      # Меняем яркость
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.7, 1.0)) # ГЛАВНОЕ: зум и обрезка
])

In [27]:
X = []
y = []

video_ext = (".mp4", ".avi", ".mov", ".mkv")

for class_name in sorted(os.listdir(DATASET_PATH)):
    class_path = os.path.join(DATASET_PATH, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"→ Класс {class_name}")

    for fname in sorted(os.listdir(class_path)):
        if not fname.lower().endswith(video_ext):
            continue

        video_path = os.path.join(class_path, fname)
        print(f"    Видео: {fname}")

        cap = cv2.VideoCapture(video_path)
        frame_id = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # 1. ТЕПЕРЬ БЕРЕМ КАЖДЫЙ 2-й КАДР (в 5 раз больше данных, чем было)
            if frame_id % 2 == 0:
                
                # --- БЛОК АУГМЕНТАЦИИ ---
                # Создаем копию кадра, чтобы не портить оригинал для следующих шагов
                aug_frame = frame.copy()

                # А. Рандомное отражение (слева-направо)
                # Это самое важное, чтобы модель не привыкала, что рука в одном углу
                if random.random() > 0.5:
                    aug_frame = cv2.flip(aug_frame, 1)

                # Б. Рандомный поворот (от -15 до 15 градусов)
                angle = random.uniform(-15, 15)
                h, w = aug_frame.shape[:2]
                M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
                aug_frame = cv2.warpAffine(aug_frame, M, (w, h))

                # В. Рандомное изменение яркости (чтобы не привязываться к свету)
                brightness = random.uniform(0.7, 1.3)
                aug_frame = cv2.convertScaleAbs(aug_frame, alpha=brightness, beta=0)
                
                # --- ИЗВЛЕЧЕНИЕ ВЕКТОРА ---
                # Передаем уже измененный кадр
                vec = extract_pose_vector(aug_frame)
                
                if vec is not None:
                    X.append(vec)
                    y.append(class_name)

            frame_id += 1

        cap.release()

X = np.array(X)
y = np.array(y)

print("Всего поз:", len(X))
print("Классы:", np.unique(y))


→ Класс 1
    Видео: 1_1.mov
    Видео: 1_2.mov
    Видео: 1_3.mov
    Видео: 1_4.MOV
    Видео: 1_5.mp4
→ Класс 2
    Видео: 2_1.mov
    Видео: 2_2.mov
    Видео: 2_3.mov
    Видео: 2_4.MOV
    Видео: 2_5.mp4
→ Класс 3
    Видео: 3_1.mov
    Видео: 3_2.mov
    Видео: 3_3.mov
    Видео: 3_4.MOV
    Видео: 3_5.mp4
→ Класс 4
    Видео: 4_1.mov
    Видео: 4_2.mov
    Видео: 4_3.mov
    Видео: 4_4.MOV
    Видео: 4_5.mp4
→ Класс 5
    Видео: 5_1.mov
    Видео: 5_2.mov
    Видео: 5_3.mov
    Видео: 5_4.MOV
    Видео: 5_5.mp4
→ Класс 6
    Видео: 6_1.mov
    Видео: 6_2.mov
    Видео: 6_3.mov
    Видео: 6_4.MOV
    Видео: 6_5.mp4
→ Класс 7
    Видео: 7_1.mov
    Видео: 7_2.mov
    Видео: 7_3.mov
    Видео: 7_4.MOV
    Видео: 7_5.mp4
→ Класс 8
    Видео: 8_1.mov
    Видео: 8_2.mov
    Видео: 8_3.mov
    Видео: 8_4.MOV
    Видео: 8_5.mp4
Всего поз: 5000
Классы: ['1' '2' '3' '4' '5' '6' '7' '8']


In [28]:
if len(np.unique(y)) < 2:
    raise ValueError(f"Нашли только один класс: {np.unique(y)}. Проверь структуру папок и данные.")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# PCA: сохраняем 98% дисперсии для лучшего качества
pca = PCA(n_components=0.98)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
print(f"PCA: {X_train_scaled.shape[1]} -> {X_train_pca.shape[1]} компонент (98% дисперсии)")


PCA: 126 -> 24 компонент (98% дисперсии)


In [29]:
# Лучшие параметры найдены экспериментально: C=200, gamma=0.05
clf = SVC(C=200, gamma=0.05, kernel='rbf', probability=True)
clf.fit(X_train_pca, y_train)

y_pred = clf.predict(X_test_pca)
print("Отчёт по качеству:\n")
print(classification_report(y_test, y_pred))
print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))


Отчёт по качеству:

              precision    recall  f1-score   support

           1       0.89      0.87      0.88       169
           2       0.96      0.94      0.95       101
           3       0.90      0.88      0.89       151
           4       1.00      0.90      0.95        61
           5       0.92      0.89      0.90       122
           6       0.89      0.91      0.90       148
           7       0.96      0.94      0.95       170
           8       0.73      0.92      0.82        78

    accuracy                           0.91      1000
   macro avg       0.91      0.91      0.91      1000
weighted avg       0.91      0.91      0.91      1000

Матрица ошибок:
[[147   1   5   0   0   8   0   8]
 [  4  95   0   0   0   0   0   2]
 [  3   1 133   0   5   2   3   4]
 [  0   0   0  55   0   0   0   6]
 [  3   1   8   0 108   0   1   1]
 [  7   0   0   0   2 135   2   2]
 [  0   0   1   0   2   4 160   3]
 [  2   1   0   0   0   2   1  72]]


In [30]:
model = {
    "clf": clf,
    "scaler": scaler,
    "pca": pca,
    "classes": sorted(list(np.unique(y)))
}

model_path = os.path.join(DATASET_PATH, "model.pkl")
with open(model_path, "wb") as f:
    pickle.dump(model, f)

print("Модель сохранена в:", model_path)


Модель сохранена в: ./data/model.pkl
